In [2]:
import pandas as pd
import os
import dotenv
import ipaddress

dotenv.load_dotenv()


True

In [3]:
data_dir = os.getenv("DATA_DIR", './data')

# Data Loading

In [4]:
private_networks = [
    ipaddress.ip_network('192.168.0.0/16'),
    ipaddress.ip_network('10.0.0.0/8'),
    ipaddress.ip_network('172.16.0.0/12')
]

def is_private_ip(ip):
    ip = ipaddress.ip_address(ip)
    return any(ip in network for network in private_networks)

def load_df(filename, dirs):
    dirs.append(filename)
    label_flow_path = os.path.join(data_dir, *dirs)
    label_flow_df = pd.read_csv(label_flow_path)
    label_flow_df.columns = label_flow_df.columns.str.strip()
    return label_flow_df

def load_label_flow_df(filename):
    dirs = ['GeneratedLabelledFlows', 'TrafficLabelling']
    label_flow_df = load_df(filename, dirs)
    label_flow_df['Source IP is Private'] = label_flow_df['Source IP'].apply(is_private_ip)
    label_flow_df['Destination IP is Private'] = label_flow_df['Destination IP'].apply(is_private_ip)
    return label_flow_df

def load_ml_df(filename):
    dirs = ['MachineLearningCSV', 'MachineLearningCVE']
    return load_df(filename, dirs)


In [5]:
label_flow_df = load_label_flow_df('Monday-WorkingHours.pcap_ISCX.csv')
ml_df = load_ml_df('Monday-WorkingHours.pcap_ISCX.csv')

# Data Exploration

## Label Flow

In [6]:
pd.set_option('display.max_columns', None)
label_flow_df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Source IP is Private,Destination IP is Private
0,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,03/07/2017 08:55:58,4,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4.0,4.0,4.0,4.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,5.000000e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,False,True
1,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,03/07/2017 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,False,True
2,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,03/07/2017 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,False,True
3,192.168.10.5-8.254.250.126-49188-80-6,8.254.250.126,80,192.168.10.5,49188,6,03/07/2017 08:55:58,1,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,2.000000e+06,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,False,True
4,192.168.10.14-8.253.185.121-49486-80-6,8.253.185.121,80,192.168.10.14,49486,6,03/07/2017 08:56:22,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,40,0,6.666667e+05,0.0,6.0,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN,False,True


In [7]:
label_flow_df.shape

(529918, 87)

In [8]:
label_flow_df.info(max_cols = None)

<class 'pandas.DataFrame'>
RangeIndex: 529918 entries, 0 to 529917
Data columns (total 87 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Flow ID                      529918 non-null  str    
 1   Source IP                    529918 non-null  str    
 2   Source Port                  529918 non-null  int64  
 3   Destination IP               529918 non-null  str    
 4   Destination Port             529918 non-null  int64  
 5   Protocol                     529918 non-null  int64  
 6   Timestamp                    529918 non-null  str    
 7   Flow Duration                529918 non-null  int64  
 8   Total Fwd Packets            529918 non-null  int64  
 9   Total Backward Packets       529918 non-null  int64  
 10  Total Length of Fwd Packets  529918 non-null  float64
 11  Total Length of Bwd Packets  529918 non-null  float64
 12  Fwd Packet Length Max        529918 non-null  float64
 13  Fwd Packet

In [9]:
label_flow_df.describe()

/home/kilnaar/anaconda3/envs/data-exploration-cic-ids-2017/lib/python3.12/site-packages/pandas/core/nanops.py:1020: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
/home/kilnaar/anaconda3/envs/data-exploration-cic-ids-2017/lib/python3.12/site-packages/pandas/core/nanops.py:1020: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
count,529918.000000,529918.000000,529918.000000,5.299180e+05,529918.000000,529918.000000,5.299180e+05,5.299180e+05,529918.000000,529918.000000,529918.000000,529918.000000,529918.000000,529918.000000,529918.000000,529918.000000,5.298540e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,529918.000000,529918.0,529918.0,529918.0,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,529918.000000,529918.000000,529918.000000,529918.000000,5.299180e+05,529918.000000,529918.000000,529918.000000,529918.000000,529918.00000,529918.000000,529918.0,529918.000000,529918.000000,529918.000000,529918.000000,529918.000000,5.299180e+05,529918.0,529918.0,529918.0,529918.0,529918.0,529918.0,529918.000000,5.299180e+05,529918.000000,5.299180e+05,529918.000000,529918.000000,529918.000000,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05,5.299180e+05
mean,40328.592461,10644.367112,10.649882,1.038927e+07,10.390315,11.517105,5.324195e+02,1.789841e+04,190.897188,20.277279,50.744078,57.452269,406.073268,49.924381,164.769337,131.309689,inf,inf,1.069388e+06,1.540446e+06,4.323909e+06,2.936914e+05,1.009897e+07,2.115864e+06,1.045617e+06,4.174560e+06,1.466088e+06,9.134728e+06,1.817322e+06,8.093950e+05,3.372632e+06,1.245437e+06,0.064138,0.0,0.0,0.0,-1.320351e+04,-3.055556e+03,6.023015e+04,6.009861e+03,19.652797,447.308521,102.624413,135.201905,7.389834e+04,0.022947,0.064138,0.000136,0.248627,0.30650,0.119992,0.0,0.000138,0.638903,116.613731,50.744078,164.769337,-1.320351e+04,0.0,0.0,0.0,0.0,0.0,0.0,10.390315,5.324195e+02,11.517105,1.789841e+04,9079.723967,3055.875568,7.412509,-3.614576e+03,6.843482e+04,4.321930e+04,1.453907e+05,4.380369e+04,3.463918e+06,2.024408e+05,3.620657e+06,3.274066e+06
std,24391.525367,21390.213475,5.439519,2.875195e+07,892.412791,1173.318788,6.228642e+03,2.675470e+06,448.833754,36.275793,91.964713,146.518081,785.038089,74.804308,276.987743,275.848232,NaN,NaN,5.113239e+06,6.057313e+06,1.402750e+07,3.883156e+06,2.858167e+07,1.024547e+07,3.911767e+06,1.397196e+07,1.002922e+07,2.768821e+07,9.714902e+06,3.598336e+06,1.290213e+07,9.456427e+06,0.244999,0.0,0.0,0.0,3.609256e+06,6.209914e+05,2.289737e+05,3.539694e+04,24.999796,840.435958,160.214664,235.836578,2.099224e+05,0.149735,0.244999,0.011656,0.432218,0.46104,0.324953,0.0,0.011736,0.552689,166.166659,91.964713,276.987743,3.609256e+06,0.0,0.0,0.0,0.0,0.0,0.0,892.412791,6.228642e+03,1173.318788,2.675470e+06,18175.305237,10137.092147,851.762351,5.526328e+05,5.872322e+05,3.971455e+05,1.028606e+06,4

In [10]:
label_flow_df['Label'].value_counts()

Label
BENIGN    529918
Name: count, dtype: int64

In [11]:
label_flow_df['Source IP'].value_counts()

Source IP
192.168.10.3     67774
192.168.10.25    58199
192.168.10.5     48656
192.168.10.12    36517
192.168.10.9     34341
                 ...  
192.243.232.9        1
65.52.108.11         1
52.84.143.21         1
91.189.88.149        1
149.56.76.152        1
Name: count, Length: 8239, dtype: int64

In [12]:
label_flow_df['Destination IP'].value_counts()

Destination IP
192.168.10.3       152194
192.168.10.1        67016
192.168.10.25       38834
192.168.10.5        14034
192.168.10.12        8772
                    ...  
209.15.238.230          1
5.79.102.231            1
23.208.157.252          1
17.253.20.125           1
151.101.192.133         1
Name: count, Length: 9698, dtype: int64

In [13]:
label_flow_df['Source IP is Private'].value_counts()

Source IP is Private
True     426184
False    103734
Name: count, dtype: int64

In [14]:
label_flow_df['Destination IP is Private'].value_counts()

Destination IP is Private
True     331766
False    198152
Name: count, dtype: int64

In [15]:
label_flow_df['Source Port'].value_counts()

Source Port
443      84887
80       18518
123       5180
137       1510
22        1023
         ...  
12851        1
17847        1
19660        1
7677         1
24323        1
Name: count, Length: 52723, dtype: int64

In [16]:
label_flow_df['Destination Port'].value_counts()

Destination Port
53       214674
443      140952
80        50834
123        5180
22         2132
          ...  
34497         1
6217          1
36236         1
40299         1
60922         1
Name: count, Length: 30238, dtype: int64

## Machine Learning

In [17]:
pd.set_option('display.max_columns', None)
ml_df.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49188,4,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,3000000.0,5.000000e+05,4.0,0.0,4,4,4,4.0,0.0,4,4,0,0.0,0.0,0,0,0,0,0,0,40,0,5.000000e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,49188,1,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,12000000.0,2.000000e+06,1.0,0.0,1,1,1,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0,0,0,40,0,2.000000e+06,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,329,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,49486,3,2,0,12,0,6,6,6.0,0.0,0,0,0.0,0.0,4000000.0,6.666667e+05,3.0,0.0,3,3,3,3.0,0.0,3,3,0,0.0,0.0,0,0,0,0,0,0,40,0,6.666667e+05,0.0,6,6,6.0,0.0,0.0,0,0,0,0,1,1,0,0,0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,245,-1,1,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [18]:
ml_df.shape

(529918, 79)